# MLFlow

mlflow server --host 127.0.0.1 --port 8080

## Quick Start

In [ ]:
import os

os.chdir("../")

from src.ollama_connection import llama_index_ollama

model = "gpt-oss:120b-cloud"

ollama_llm = llama_index_ollama(model=model, temperature=0)

In [ ]:
import mlflow

# 通知mlflow要把紀錄送去哪裡
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Create a new MLflow Experiment
mlflow.set_experiment("LlamaIndex")

快速的使用第一周的範例建立vectorstoreindex

讀取檔案

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext, Document 
from llama_index.readers.file import PyMuPDFReader

loader = PyMuPDFReader()

docs = loader.load("week_1/08物理.pdf")

doc_text = "\n\n".join([d.get_content() for d in docs])
docs = [Document(text=doc_text)]

建立VectorStoreIndex

In [ ]:
import faiss
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SimpleFileNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")
d = 1024 # 必須與 embedding model 的輸出維度一致

faiss_index = faiss.IndexFlatL2(d)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    [],
    storage_context=storage_context,
    transformations=[SimpleFileNodeParser()],
    embed_model=embed_model
)

# add documents to index
for doc in docs:
    index.insert(doc)

In [ ]:
query_engine = index.as_query_engine(
    similarity_top_k=5,
    llm=ollama_llm,
    embed_model=embed_model,
)

Tracing

In [ ]:
mlflow.llama_index.autolog()

In [ ]:
response = await query_engine.aquery("基本力學的學習目標為何?")

log model

In [ ]:
with mlflow.start_run():
    model_info = mlflow.llama_index.log_model(
        index,
        name="index",
        engine_type="query",
        input_example="基本粒子的學習目標",
        registered_model_name="llama_index_basic" # 登記模型名稱
    )

這種方法可以順利的log_model，但是load_model會出問題。因為外部vectorstore像是FAISS要讀取的是binary格式檔案

所以繞過的方法是建立一個python file，然後把模型寫在那個檔案裡

參照index.py

In [ ]:
with mlflow.start_run():
    model_info = mlflow.llama_index.log_model(
        "week_5/index.py",
        name="index",
        engine_type="query",
        input_example="基本粒子的學習目標",
        registered_model_name="llama_index_basic" # 登記模型名稱
    )

In [ ]:
index_loaded = mlflow.llama_index.load_model(model_uri="models:/llama_index_basic/1")

In [ ]:
index_loaded.as_query_engine(llm=ollama_llm).query("電磁學的學習目標")

Multi-file dependencies

In [ ]:
with mlflow.start_run():
    model_info = mlflow.llama_index.log_model(
        "week_5/index_multi.py",
        name="index",
        engine_type="query",
        input_example="熱力學的學習目標",
        code_paths=["week_5/load_index.py", "week_5/load_model.py"],  # Include dependency
        registered_model_name="llama_index_basic" # 登記模型名稱
    )

In [ ]:
index_loaded = mlflow.llama_index.load_model(model_uri="models:/llama_index_basic/2")

In [ ]:
index_loaded.as_query_engine(llm=ollama_llm).query("電磁學的學習目標")

模擬遠端執行

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# 列出特定模型的所有版本
versions = client.search_model_versions("name='llama_index_basic'")

for v in versions:
    print(f"Version: {v.version}, Stage: {v.current_stage}, Run ID: {v.run_id}")

給予模型Alias

In [ ]:
client.set_registered_model_alias(
    name="llama_index_basic",
    alias="champion",
    version="1"
)

### 模型描述

In [ ]:
client.update_registered_model(
    name="llama_index_basic",
    description="Basic LlamaIndex model for document Q&A"
)

### 模型 Tags（標籤）

MLflow 支援兩層 Tag：

- Registered model-level tags：整體模型的 metadata，例如用途、團隊等資訊。

- Model version-level tags：針對每個版本做不同註記，例如驗證狀態、效能資訊等。

In [ ]:
client.set_registered_model_tag(
    name="llama_index_basic",
    key="framework",
    value="llama-index"
)

client.set_registered_model_tag(
    name="llama_index_basic",
    key="owner",
    value="nlp-team"
)

給予特定模型版本 description 和 tag

In [ ]:
client.update_model_version(
    name="llama_index_basic",
    version="1",
    description="Uses OpenAI embeddings + FAISS vector store"
)

client.set_model_version_tag(
    name="llama_index_basic",
    version="1",
    key="embedding_model",
    value="text-embedding-3-large"
)

client.set_model_version_tag(
    name="llama_index_basic",
    version="1",
    key="index_type",
    value="faiss"
)

Get model version via alias

You cannot pass an alias directly into update_model_version() or set_model_version_tag().

Those APIs require an explicit version number.

So the workflow is:

Alias → resolve to version → update that version

In [ ]:
mv = client.get_model_version_by_alias(
    name="llama_index_basic",
    alias="champion")

In [ ]:
mv.version

In [ ]:
model_uri = "models:/llama_index_basic/1"

index_remote = mlflow.pyfunc.load_model(model_uri)

In [ ]:
model_info.registered_model_name

In [ ]:
mlflow.set_logged_model_tags(
        model_info.model_id, {"Training Info": "Basic LR model for iris data"}
    )
    